# Start Up

Run the cells below. They will open a browser window through selenium. (check your open windows if you don't notice it. It will be a firefox page with an orange url bar.)

Log into your ESPN account and navigate to the draft.

Make sure to keep everything in a single tab. Do not open links as new windows, do not open links as new tabs. This should be the default behavior if you just click on things normally (but no promises - this is hacked together at best...)

### Updating the probabilities

The default score/pick probability info is for 10 team, standard (non-ppr) scoring.

You can update the probabilities with different parameters ```yahoowizard.create_prob_pickle(teams, outputname, mapping_path, scoring, outputpath)``` where teams is the number of teams in your league, and scoring is one of these four entries:
 - 'standard'
 - 'ppr'
 - 'half-ppr'
 - '2qb

The other parameters deal with file mappings.
 
Follow this link to see the source data [Fantasy Football Calculator](https://fantasyfootballcalculator.com/scenario-calculator)

### Updating/Changing the Projections

You can call ```yahoowizard.process_projections(projection_path, idmap_path, output_path)```
to update or change the projections. ```projection_path``` is the direction to the projection files. I use [fantasyfootballanalytics.net](https://www.fantasyfootballanalytics.net)

You can use any source of projections as long as you can map them to the ESPN IDs. 

Required columns:
 - 'position'
 - 'player'
 - 'points'
 - 'adp'
 
This needs more development, specifically:
 - mapping tables to cross-reference all the different sources to some sort of master ID
 - It would also be useful to map the player IDs to the different providers.
 - making the column naming more flexible 
 
 
 # Current Use.
 tells you how to use it... tbd

In [1]:
import pandas as pd 
import numpy as np
import yahoowizard as yw

[2020-09-02 15:32:34,769 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2020-09-02 15:32:34,773 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2527.8708856105804
[2020-09-02 15:32:34,775 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2020-09-02 15:32:34,778 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2527.8758721351624
[2020-09-02 15:32:34,780 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


# projections

In [2]:
proj_name = 'yahoo_players.pkl'
prob_name = 'pick_prob_NEW_10_standard.pkl'

yw.process_projections('ffa_customrankings2020-0.csv',
                       'espn_ffa_map.csv',
                       proj_name)

# yw.update_probabilities(10,
#                         'standard',
#                         prob_name,
#                         'probmap.csv')

projections = pd.read_pickle(proj_name)

probabilities = pd.read_pickle(prob_name)



[2020-09-02 15:32:34,856 INFO] [numexpr.utils.utils._init_num_threads] NumExpr defaulting to 8 threads.


In [3]:
projections.columns

Index(['playerId', 'player', 'team', 'position', 'age', 'exp', 'bye', 'points',
       'lower', 'upper', 'sdPts', 'positionRank', 'dropoff', 'tier',
       'ptSpread', 'overallECR', 'positionECR', 'sdRank', 'risk', 'sleeper',
       'vor', 'overallRank', 'vorLow', 'vorHigh', 'adp', 'auctionValue',
       'adpDiff', 'cost', 'ppdollar', 'espn_names', 'espn_id', 'picked',
       'blacklist', 'oc_adj', 'player_id', 'stats_id', 'yahoo_id',
       'rotowire_id', 'rotoworld_id', 'fantasy_data_id', 'sportradar_id'],
      dtype='object')

In [4]:
league_id = 9989340
your_first_pick = 6

In [5]:
draft = yw.YahooDraft(league_id, your_first_pick)

In [6]:
draft.league_id

'9989340'

In [7]:
draft.first_pick

6

In [8]:
draft.current_pick

1

In [9]:
draft.num_teams

10

In [10]:
draft.req_positions.sum()

15

In [19]:
oc = yw.tell_me_what_to_do(projections,probabilities,draft)
cols = ['player','team','position','points','adp','oc_raw_0','oc_adj_0','oc_adj_1']
print('\nRoster Needs')
print(draft.pos_needs.loc[['QB','RB','WR','TE','D/ST','K']])
print('\n')
oc[cols].sort_values('oc_adj_1',ascending=False).head(10)

this pick is 55
Your Next Pick is 66
Your 2nd Pick is 75

Roster Needs
       slot   flex   need
QB     True  False   True
RB    False  False  False
WR     True  False   True
TE    False  False  False
D/ST   True  False   True
K      True  False   True




,player,team,position,points,adp,oc_raw_0,oc_adj_0,oc_adj_1
34,David Montgomery,CHI,RB,157.738674,44.5733,27.532893,13.766447,18.064908
44,D'Andre Swift,DET,RB,146.748735,57.5750,12.969676,6.484838,12.318539
59,Kareem Hunt,CLE,RB,137.901553,77.4033,6.880020,3.440010,8.105615
35,Deshaun Watson,HOU,QB,294.993798,56.3933,8.993886,2.698166,5.116341
56,Tyler Lockett,SEA,WR,151.476304,53.8167,0.322334,0.161167,4.577192
53,DeVante Parker,MIA,WR,152.464693,59.3450,0.120616,0.060308,2.814620
72,Darren Waller,LVR,TE,118.040437,60.0900,3.250805,0.975242,2.383343
47,Kyler Murray,ARI,QB,283.821322,71.3267,1.444956,0.433487,2.010874
67,T.Y. Hilton,IND,WR,144.640913,63.2733,-6.278276,-3.139138,1.159497
69,D.J. Chark,JAC,WR,144.162572,58.2400,-6.795325,-3.397663,0.920326


In [13]:
projections = yw.blacklist_player('Leonard Fournette','JAC',projections)
# projections = yw.unblacklist_player('Leonard Fournette','JAC',projections)

Blacklisting these players:
               player team position
19  Leonard Fournette  JAC       RB
